# Verify the ProprioTunedCNN sensorprocessing

Reload a pre-trained model specified by an experiment trained by the Train-ProprioTuned-CNN notebook.

This notebook runs a number of experiments that illustrate the performance. 

In [1]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from behavior_cloning.demo_to_trainingdata import BCDemonstration
from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, VGG19ProprioTunedSensorProcessing
from robot.al5d_position_controller import RobotPosition

# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
run = "vgg19_128"
exp = Config().get_experiment(experiment, run)

Loading pointer config file: C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
Note: no system dependent config file G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\vgg19_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_propriotuned_cnn/vgg19_128 successfully loaded


### Check the model for an encoding mode

Check whether we can load back the model

In [3]:
exp = Config().get_experiment(experiment, run)
hidden_size = exp["latent_dims"]
output_size = Config()["robot"]["action_space_size"]

if exp['model'] == 'VGG19ProprioTunedRegression':
    enc = VGG19ProprioTunedRegression(
        hidden_size=hidden_size, output_size=output_size)
else:
    raise Exception(f"Unknown model {exp['model']}")

modelfile = pathlib.Path(exp["data_dir"], 
                         exp["proprioception_mlp_model_file"])
assert modelfile.exists()
enc.load_state_dict(torch.load(modelfile))

Note: no system dependent config file G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\vgg19_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_propriotuned_cnn/vgg19_128 successfully loaded


c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\lboloni\AppData\Local\Temp\ipykernel_159116\1255638504.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute a

<All keys matched successfully>

In [4]:
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
enc = enc.to(device)

inputlist = []
targetlist = []
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            z = enc.encode(sensor_readings)
            break

print(z)
print(z.shape)


Cameras found: ['dev2']
There are 388 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
tensor([[-4.5129e+00, -2.2058e+00, -3.2404e+00,  7.5834e-01,  3.0324e+00,
          4.2052e+00, -1.1299e+00, -8.0757e+00, -6.0706e+00, -7.1044e+00,
         -6.0594e+00, -3.7787e+00,  8.4719e+00, -1.0134e+00, -2.8241e+00,
          2.4231e+00, -7.3312e+00, -4.4464e+00, -5.6492e+00,  3.2451e+00,
          2.6959e-01, -4.2067e+00, -3.7391e+00, -5.5489e+00,  2.8673e+00,
         -5.5216e+00, -1.9480e+00,  1.0107e+00,  1.2318e-01, -4.0668e+00,
         -7.1592e+00,  9.1650e-01,  4.9050e+00,  4.7074e+00,  3.3768e+00,
         -5.6155e+00, -3.7511e+00, -1.3502e+00,  1.9216e+00, -8.1680e+00,
          1.4490e+00, -1.0846e+00,  5.5941e-01, -4.3811e+00, -2.8466e+00,
          5.3295e+00,  1.9601e+00,  4.2959e-01, -1.9975e+00, -7.5016e+00,
         -2.9689e-01, -1.2750e-01, -1.2100e+00,  3.5364e+00, -4.7736e+00,
         -4.3391e+00,  2.8574e+00,  2.2698e+00, -3.60

### Show how to use an SensorProcessing object

In [5]:
# Create the sensor processing package

exp = Config().get_experiment("sensorprocessing_propriotuned_cnn", run)
sp = VGG19ProprioTunedSensorProcessing(exp, device=device)

Note: no system dependent config file G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\vgg19_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_propriotuned_cnn/vgg19_128 successfully loaded


c:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\sensorprocessing\..\sensorprocessing\sp_propriotuned_cnn.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self

In [6]:
# Run the sensor processing package to process the images of a demonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            sensor_readings = sensor_readings.to(device)
            z = sp.process(sensor_readings)
            break

print(z)
print(z.shape)

Cameras found: ['dev2']
There are 388 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
sensor readings shape torch.Size([1, 3, 256, 256])
[-4.5129495e+00 -2.2057865e+00 -3.2404022e+00  7.5834453e-01
  3.0324214e+00  4.2052059e+00 -1.1298878e+00 -8.0757351e+00
 -6.0705962e+00 -7.1044106e+00 -6.0594215e+00 -3.7787216e+00
  8.4719229e+00 -1.0133907e+00 -2.8240857e+00  2.4230831e+00
 -7.3312058e+00 -4.4464378e+00 -5.6491523e+00  3.2450981e+00
  2.6958919e-01 -4.2066574e+00 -3.7390933e+00 -5.5489392e+00
  2.8673432e+00 -5.5216498e+00 -1.9480151e+00  1.0106723e+00
  1.2317884e-01 -4.0667801e+00 -7.1591649e+00  9.1649717e-01
  4.9049687e+00  4.7073622e+00  3.3768213e+00 -5.6155210e+00
 -3.7511401e+00 -1.3501763e+00  1.9216290e+00 -8.1679592e+00
  1.4490038e+00 -1.0846056e+00  5.5941057e-01 -4.3810587e+00
 -2.8465586e+00  5.3295422e+00  1.9601411e+00  4.2959088e-01
 -1.9975140e+00 -7.5015874e+00 -2.9688755e-01 -1.2750435e-01
 -1.2100486e+00  3.5363

In [7]:
# Run the sensor processing inside the BCDemonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=sp)
        for i in range(bcd.trim_from, bcd.trim_to):
            #sensor_readings, _ = bcd.get_image(i)
            #sensor_readings = sensor_readings.to(device)
            #z = sp.process(sensor_readings)
            z = bcd.get_z(i)
            break

print(z)
print(z.shape)

Cameras found: ['dev2']
There are 388 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
sensor readings shape torch.Size([1, 3, 256, 256])
[-4.5129495e+00 -2.2057865e+00 -3.2404022e+00  7.5834453e-01
  3.0324214e+00  4.2052059e+00 -1.1298878e+00 -8.0757351e+00
 -6.0705962e+00 -7.1044106e+00 -6.0594215e+00 -3.7787216e+00
  8.4719229e+00 -1.0133907e+00 -2.8240857e+00  2.4230831e+00
 -7.3312058e+00 -4.4464378e+00 -5.6491523e+00  3.2450981e+00
  2.6958919e-01 -4.2066574e+00 -3.7390933e+00 -5.5489392e+00
  2.8673432e+00 -5.5216498e+00 -1.9480151e+00  1.0106723e+00
  1.2317884e-01 -4.0667801e+00 -7.1591649e+00  9.1649717e-01
  4.9049687e+00  4.7073622e+00  3.3768213e+00 -5.6155210e+00
 -3.7511401e+00 -1.3501763e+00  1.9216290e+00 -8.1679592e+00
  1.4490038e+00 -1.0846056e+00  5.5941057e-01 -4.3810587e+00
 -2.8465586e+00  5.3295422e+00  1.9601411e+00  4.2959088e-01
 -1.9975140e+00 -7.5015874e+00 -2.9688755e-01 -1.2750435e-01
 -1.2100486e+00  3.5363